#!/usr/bin/env python
"""
NH3 dry + wet deposition (monthly) • IASI NH3 + ERA5 drivers
"""

# ------------------------------------------------------------------
# 0.  Imports & constants
# ------------------------------------------------------------------
import xarray as xr, numpy as np, pandas as pd, pathlib, warnings, xesmf as xe
warnings.filterwarnings("ignore", category=RuntimeWarning)

NH3_FILE  = "NH3_Interpolated_IASI_AB_201901_202205_merged_satellite.nc"
ERA_FILE  = "era5_drv_wetdry_2019-2022.nc"
OUT_FILE  = "NH3_dry_wet_dep_2019-2022.nc"

MW_NH3    = 17.031           # g mol-1   [oai_citation:6‡codes.ecmwf.int](https://codes.ecmwf.int/grib/param-db/228003?utm_source=chatgpt.com)
AVOGADRO  = 6.02214076e23    # mol-¹     [oai_citation:7‡NIST](https://physics.nist.gov/cgi-bin/cuu/Value?na=&utm_source=chatgpt.com)
CM2_TO_M2 = 1e4              # 1 cm² → 1×10⁻⁴ m²
LAMBDA    = 2.0e-5           # s-¹ (mm h-¹)⁻¹   [oai_citation:8‡ScienceDirect](https://www.sciencedirect.com/science/article/abs/pii/S1352231008004949?utm_source=chatgpt.com)
VD_A, VD_B = 0.001, 0.005    # Vd = 0.1 + 0.5·u*  (m s-¹)   [oai_citation:9‡ScienceDirect](https://www.sciencedirect.com/science/article/abs/pii/S135223100800469X?utm_source=chatgpt.com)

# ------------------------------------------------------------------
# 1.  Load monthly datasets
# ------------------------------------------------------------------
nh3 = xr.open_dataset(NH3_FILE)            # dims: time, lat, lon
era = xr.open_dataset(ERA_FILE)            # same monthly cadence

# 1.1  Convert NH3 column → kg m-2
col_mol = nh3.NH3 * 1e16 * CM2_TO_M2 / AVOGADRO
mass = col_mol * MW_NH3 / 1000             # kg m-2
mass.name = "NH3_mass"

# ------------------------------------------------------------------
# 2.  Re-grid ERA5 → NH3 grid if needed
# ------------------------------------------------------------------
if era.dims["lat"] != nh3.dims["lat"] or era.dims["lon"] != nh3.dims["lon"]:
    print("• regridding ERA5 to NH3 grid (conservative)…")
    regrid = xe.Regridder(era, nh3, "conservative", reuse_weights=True)  #  [oai_citation:10‡xesmf.readthedocs.io](https://xesmf.readthedocs.io/en/latest/notebooks/Compare_algorithms.html?utm_source=chatgpt.com) [oai_citation:11‡coecms-training.github.io](https://coecms-training.github.io/parallel/case-studies/regridding.html?utm_source=chatgpt.com)
    era = regrid(era)

# ------------------------------------------------------------------
# 3.  Prepare helper arrays
# ------------------------------------------------------------------
days_in_month = era.time.dt.days_in_month
hours_in_month = days_in_month * 24
seconds_in_month = days_in_month * 86400

# Rain rate mm h-1  (tp is m month-1 → mm month-1 / hours)
rain_rate = era.tp * 1_000 / hours_in_month
lambda_p  = LAMBDA * rain_rate            # s-¹

# Deposition velocity (m s-¹)
Vd = VD_A + VD_B * era.zust

# Near-surface concentration (kg m-3)
Csurf = mass / era.blh

# ------------------------------------------------------------------
# 4.  Fluxes → monthly totals
# ------------------------------------------------------------------
flux_dry = Vd * Csurf                     # kg m-2 s-1
flux_wet = lambda_p * mass               # kg m-2 s-1

dry_dep = (flux_dry * seconds_in_month).transpose("time", "lat", "lon")
wet_dep = (flux_wet * seconds_in_month).transpose("time", "lat", "lon")

dry_dep.name = "NH3_dry_dep"
wet_dep.name = "NH3_wet_dep"
for da in (dry_dep, wet_dep):
    da.attrs["units"] = "kg NH3 m⁻² month⁻¹"

# ------------------------------------------------------------------
# 5.  Package & save
# ------------------------------------------------------------------
out = xr.Dataset(
        {"NH3_mass": mass,
         "NH3_dry_dep": dry_dep,
         "NH3_wet_dep": wet_dep},
        coords=nh3.coords
)
out.to_netcdf(OUT_FILE)
print("✓ wrote", OUT_FILE)

#!/usr/bin/env python
"""
NH₃ dry + wet deposition (monthly)  •  fallback to xarray.interp if xESMF/ESMF missing
-------------------------------------------------------------------------------------
"""

import xarray as xr, numpy as np, pandas as pd, warnings, importlib

NH3_FILE = "NH3_Interpolated_IASI_AB_201901_202205_merged_satellite.nc"
ERA_FILE = "era5_drv_wetdry_2019-2022.nc"
OUTFILE  = "NH3_dry_wet_dep_2019-2022.nc"

# ---- physical constants ----------------------------------------------------
MW_NH3, AVO, CM2_TO_M2 = 17.031, 6.02214076e23, 1e4
LAMBDA = 2e-5          # s⁻¹ (mm h⁻¹)⁻¹   [oai_citation:8‡GitHub](https://github.com/pydata/xarray/issues/5987?utm_source=chatgpt.com)
VD_A, VD_B = 0.001, 0.005   # Vd = 0.1 + 0.5·u* (m s⁻¹)  [oai_citation:9‡ACCESS Hive Community Forum](https://forum.access-hive.org.au/t/regridding-with-land-sea-masks-advice/673?utm_source=chatgpt.com)

# ---- load monthly data ------------------------------------------------------
nh3 = xr.open_dataset(NH3_FILE)
# era = xr.open_dataset(ERA_FILE)
era = xr.open_dataset("era5_drv_wetdry_2019-2022.nc", engine="h5netcdf")
# NH₃ column → kg m⁻²
mass = (nh3.NH3 * 1e16 * CM2_TO_M2 / AVO) * MW_NH3 / 1000
mass.name = "NH3_mass"

# ---- regridding: xESMF if present, else xarray.interp -----------------------
def regrid_era(src, tgt):
    if importlib.util.find_spec("xesmf") is not None and \
       (importlib.util.find_spec("esmpy") or importlib.util.find_spec("ESMF")):
        import xesmf as xe
        regridder = xe.Regridder(src, tgt, method="conservative", reuse_weights=True)
        return regridder(src)
    else:
        warnings.warn("ESMF not found – falling back to xarray.interp (nearest)")
        return src.interp(lat=tgt.lat, lon=tgt.lon, method="nearest")

era_rg = regrid_era(era, nh3)

# ---- helper arrays ----------------------------------------------------------
days  = era_rg.time.dt.days_in_month
secs  = days * 86400
hrs   = days * 24
rain_rate = era_rg.tp * 1000 / hrs        # mm h⁻¹
lambda_p  = LAMBDA * rain_rate            # s⁻¹

# ---- dry deposition ---------------------------------------------------------
Vd    = VD_A + VD_B * era_rg.zust
Csurf = mass / era_rg.blh
dry   = (Vd * Csurf * secs).transpose("time", "lat", "lon")
dry.name = "NH3_dry_dep"; dry.attrs["units"] = "kg NH3 m⁻² month⁻¹"

# ---- wet deposition ---------------------------------------------------------
wet   = (lambda_p * mass * secs).transpose("time", "lat", "lon")
wet.name = "NH3_wet_dep"; wet.attrs["units"] = "kg NH3 m⁻² month⁻¹"

# ---- save -------------------------------------------------------------------
out = xr.Dataset({"NH3_dry_dep": dry, "NH3_wet_dep": wet}, coords=nh3.coords)
out.to_netcdf(OUTFILE)
print("✓ wrote", OUTFILE)

In [5]:
import netCDF4, h5netcdf
print(netCDF4.__version__, h5netcdf.__version__)

1.7.2 1.6.3


era = xr.open_dataset("era5_drv_wetdry_2019-2022.nc")

# if you extracted the archive:
era = xr.open_mfdataset("data_stream-oper_stepType-*.nc", combine="by_coords")

# OR if you re-downloaded a single NetCDF file:
era = xr.open_dataset("era5_drv_wetdry_2019-2022.nc")

#!/usr/bin/env python
"""NH₃ dry + wet deposition using ERA5 instant & accum NetCDFs (monthly)."""

import xarray as xr, glob, zipfile, tempfile, importlib, warnings, numpy as np

# ------------------------------------------------------------------ #
# 1.  FILE PATHS                                                     #
# ------------------------------------------------------------------ #
NH3_FILE = "NH3_Interpolated_IASI_AB_201901_202205_merged_satellite.nc"
ERA_CONTAINER = "era5_drv_wetdry_2019-2022.zip"   # the *.zip* masquerading as *.nc*
OUTFILE = "NH3_dry_wet_dep_2019-2022.nc"

# ------------------------------------------------------------------ #
# 2.  CONSTANTS                                                      #
# ------------------------------------------------------------------ #
MW, AVO, CM2_M2 = 17.031, 6.02214076e23, 1e4          # kg, mol-1 etc.
LAMBDA = 2e-5                                         # s-1 (mm h-1)-1  [oai_citation:2‡b.tellusjournals.se](https://b.tellusjournals.se/articles/10.3402/tellusb.v41i3.15082?utm_source=chatgpt.com)
VD_A, VD_B = 0.001, 0.005                             # Vd = 0.1 + 0.5 u* cm s-1 → m s-1  [oai_citation:3‡ACP](https://acp.copernicus.org/articles/3/2067/2003/?utm_source=chatgpt.com) [oai_citation:4‡ScienceDirect](https://www.sciencedirect.com/science/article/pii/0004698189901534?utm_source=chatgpt.com)

# ------------------------------------------------------------------ #
# 3.  OPEN ERA5  (unzips on the fly, then merges by coords)          #
# ------------------------------------------------------------------ #
def open_era_zip(container):
    if zipfile.is_zipfile(container):
        tmp = tempfile.TemporaryDirectory()
        with zipfile.ZipFile(container) as z:
            z.extractall(tmp.name)
        files = glob.glob(f"{tmp.name}/*.nc")           # instant + accum
    else:
        files = [container]
    return xr.open_mfdataset(files, combine="by_coords") # auto-stitch  

era = open_era_zip(ERA_CONTAINER)
nh3 = xr.open_dataset(NH3_FILE)

# ------------------------------------------------------------------ #
# 4.  OPTIONAL REGRID (conservative if xESMF+ESMF are present)       #
# ------------------------------------------------------------------ #
def regrid_like(src, tgt):
    if src.dims == tgt.dims and np.allclose(src.lat, tgt.lat) and np.allclose(src.lon, tgt.lon):
        return src
    if importlib.util.find_spec("xesmf") and importlib.util.find_spec("esmpy"):
        import xesmf as xe                                            #  [oai_citation:7‡xesmf.readthedocs.io](https://xesmf.readthedocs.io/en/latest/notebooks/Compare_algorithms.html) [oai_citation:8‡xesmf.readthedocs.io](https://xesmf.readthedocs.io/en/latest/)
        return xe.Regridder(src, tgt, "conservative", reuse_weights=True)(src)
    warnings.warn("ESMF not found – falling back to nearest-neighbour")
    return src.interp(lat=tgt.lat, lon=tgt.lon, method="nearest")     #  [oai_citation:9‡cehbrecht.github.io](https://cehbrecht.github.io/jupyter-guide-to-climate-data/Regrid_xESMF.html)

era = regrid_like(era, nh3)

# ------------------------------------------------------------------ #
# 5.  NH₃ COLUMN  →  MASS PER AREA                                   #
# ------------------------------------------------------------------ #
mass = (nh3.NH3 * 1e16 * CM2_M2 / AVO) * MW / 1000    # kg m-2

# ------------------------------------------------------------------ #
# 6.  MONTHLY HELPER ARRAYS                                          #
# ------------------------------------------------------------------ #
days  = era.time.dt.days_in_month
secs  = days * 86_400
hours = days * 24
rain_rate = era.tp * 1_000 / hours                    # mm h-1  [oai_citation:10‡confluence.ecmwf.int](https://confluence.ecmwf.int/pages/viewpage.action?pageId=197702790&utm_source=chatgpt.com)
lambda_p  = LAMBDA * rain_rate

# ------------------------------------------------------------------ #
# 7.  DRY & WET FLUXES                                               #
# ------------------------------------------------------------------ #
Vd   = VD_A + VD_B * era.zust                        # m s-1   [oai_citation:11‡codes.ecmwf.int](https://codes.ecmwf.int/grib/param-db/228003?utm_source=chatgpt.com) [oai_citation:12‡ScienceDirect](https://www.sciencedirect.com/science/article/abs/pii/S1352231003005843?utm_source=chatgpt.com)
Csurf = mass / era.blh                               # kg m-3   [oai_citation:13‡codes.ecmwf.int](https://codes.ecmwf.int/grib/param-db/159?utm_source=chatgpt.com)
dry  = (Vd * Csurf * secs).transpose("time","lat","lon")
wet  = (lambda_p * mass * secs).transpose("time","lat","lon")

dry.attrs.update(units="kg NH3 m-2 month-1", long_name="Dry NH₃ deposition")
wet.attrs.update(units="kg NH3 m-2 month-1", long_name="Wet NH₃ deposition")

xr.Dataset({"NH3_dry_dep": dry, "NH3_wet_dep": wet}, coords=nh3.coords).to_netcdf(OUTFILE)
print("✓ wrote", OUTFILE)

In [5]:
#!/usr/bin/env python
"""
NH₃ dry + wet deposition • ERA5 (legacy NetCDF) + IASI NH₃ column
"""

import xarray as xr, numpy as np, importlib, warnings

# -------------------------------------------------------------------
# 1.  FILES
# -------------------------------------------------------------------
NH3_FILE = "NH3_Interpolated_IASI_AB_201901_202205_merged_satellite.nc"
ERA_FILE = "era5_drv_wetdry_2019-2022.nc"      # legacy NetCDF, not zipped
OUTFILE  = "NH3_dry_wet_dep_2019-2022.nc"

# -------------------------------------------------------------------
# 2.  CONSTANTS
# -------------------------------------------------------------------
MW_NH3   = 17.031          # g mol-1
AVO      = 6.02214076e23   # molecules mol-1
CM2_M2   = 1e4             # cm² → m²
LAMBDA   = 2e-5            # s-1 (mm h-1)-1  (below-cloud Λ)  
VD_A, VD_B = 0.001, 0.005  # Vd = 0.1 + 0.5 u* cm s-1 → m s-1  
# -------------------------------------------------------------------
# 3.  LOAD DATASETS
# -------------------------------------------------------------------
era = xr.open_dataset(ERA_FILE, engine="netcdf4")   # single file 
nh3 = xr.open_dataset(NH3_FILE)


 

# -------------------------------------------------------------------
# 4.  (OPTIONAL) REGRID ERA5 → NH₃ GRID
# -------------------------------------------------------------------
#   make the ERA5 names match the NH₃ names
era = era.rename({'latitude': 'lat', 'longitude': 'lon'}) 

# quick, dependency-free re-gridding: nearest-neighbour onto NH₃ grid
era = era.interp_like(nh3, method="nearest") 

# -------------------------------------------------------------------
# 5.  NH₃ COLUMN  →  kg m-²
# -------------------------------------------------------------------
mass = (nh3.NH3 * 1e16 * CM2_M2 / AVO) * MW_NH3 / 1000        # kg m-2  

# -------------------------------------------------------------------
# 6.  HELPER ARRAYS
# -------------------------------------------------------------------
days  = era.time.dt.days_in_month
secs  = days * 86_400
hours = days * 24
rain_rate = era.tp * 3600                            # mm h-1
lambda_p = LAMBDA * rain_rate 
# -------------------------------------------------------------------
# 7.  DRY + WET FLUXES
# -------------------------------------------------------------------
Vd   = VD_A + VD_B * era.zust                                 # m s-1    
Csurf = mass / era.blh                                        # kg m-3 
dry  = (Vd * Csurf * secs).transpose("time","lat","lon")
wet  = (lambda_p * mass * secs).transpose("time","lat","lon")

for da in (dry, wet):
    da.attrs["units"] = "kg NH₃ m⁻² month⁻¹"

xr.Dataset({"NH3_dry_dep": dry,
            "NH3_wet_dep": wet},
           coords=nh3.coords).to_netcdf(OUTFILE)

print("✓ wrote", OUTFILE)

✓ wrote NH3_dry_wet_dep_2019-2022.nc


In [6]:
#!/usr/bin/env python
"""Plot monthly NH₃ dry & wet deposition maps."""

import pathlib
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs              # CRS objects (PlateCarree, etc.)
import cartopy.feature as cfeature      # Coastlines, borders, land masks
# ------------------------------------------------------------------
# 1.  Settings you might want to tweak
# ------------------------------------------------------------------
DATAFILE = "NH3_dry_wet_dep_2019-2022.nc"
OUTDIR_DRY = pathlib.Path("fig_dry")
OUTDIR_WET = pathlib.Path("fig_wet")
COASTLINES = True        # set False if Cartopy missing / no coast wanted

# ------------------------------------------------------------------
# 2.  Load the dataset
# ------------------------------------------------------------------
ds = xr.open_dataset(DATAFILE)                # vars: NH3_dry_dep, NH3_wet_dep
dry = ds.NH3_dry_dep
wet = ds.NH3_wet_dep

# Choose a fixed colour scale (0–95 % quantile keeps extremes readable)
vmin_d, vmax_d = 0, dry.quantile(0.95).item()
vmin_w, vmax_w = 0, wet.quantile(0.95).item()

# Make sure output folders exist
OUTDIR_DRY.mkdir(exist_ok=True)
OUTDIR_WET.mkdir(exist_ok=True)

# ------------------------------------------------------------------
# 3.  Plot loop
# ------------------------------------------------------------------
for t in dry.time:
    label = str(t.dt.strftime("%Y-%m").item())        # e.g. "2019-01"

    # -------- dry deposition figure --------------------------------
    fig, ax = plt.subplots(figsize=(8,4.5), dpi=120,
                        subplot_kw={"projection": ccrs.PlateCarree()})
    pcm = dry.sel(time=t).plot(
        ax=ax, cmap="plasma", vmin=vmin_d, vmax=vmax_d,
        add_colorbar=False, robust=False
    )
    ax.set_title(f"Dry NH₃ deposition  •  {label}")
    if COASTLINES:
        import cartopy.crs as ccrs, cartopy.feature as cfeature
        ax.coastlines(linewidth=0.5)
        ax.add_feature(cfeature.BORDERS, linewidth=0.2)
    fig.colorbar(pcm, ax=ax, label=dry.attrs.get("units", "kg NH₃ m⁻² month⁻¹"))
    fig.tight_layout()
    fig.savefig(OUTDIR_DRY / f"NH3_dry_{label}.png")
    plt.close(fig)

    # -------- wet deposition figure --------------------------------
    fig, ax = plt.subplots(figsize=(8,4.5), dpi=120,
                        subplot_kw={"projection": ccrs.PlateCarree()})
    pcm = wet.sel(time=t).plot(
        ax=ax, cmap="viridis", vmin=vmin_w, vmax=vmax_w,
        add_colorbar=False, robust=False
    )
    ax.set_title(f"Wet NH₃ deposition  •  {label}")
    if COASTLINES:
        ax.coastlines(linewidth=0.5)
        ax.add_feature(cfeature.BORDERS, linewidth=0.2)
    fig.colorbar(pcm, ax=ax, label=wet.attrs.get("units", "kg NH₃ m⁻² month⁻¹"))
    fig.tight_layout()
    fig.savefig(OUTDIR_WET / f"NH3_wet_{label}.png")
    plt.close(fig)

print(f"✓  Plotted {len(dry.time)} months → {OUTDIR_DRY}/  and  {OUTDIR_WET}/")

✓  Plotted 41 months → fig_dry/  and  fig_wet/


In [8]:
#!/usr/bin/env python
"""Aggregate monthly NH₃ deposition → yearly totals and plot maps."""

import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pathlib
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# -------------------------------------------------------------------
# 1.  Paths
# -------------------------------------------------------------------
INFILE  = "NH3_dry_wet_dep_2019-2022.nc"     # monthly data you generated
OUTFILE = "NH3_yearly_dep_2019-2022.nc"      # new file to be written
ODRY    = pathlib.Path("fig_dry_yearly")
OWET    = pathlib.Path("fig_wet_yearly")
ODRY.mkdir(exist_ok=True); OWET.mkdir(exist_ok=True)

# -------------------------------------------------------------------
# 2.  Read monthly data
# -------------------------------------------------------------------
ds_mo = xr.open_dataset(INFILE)              # variables: NH3_dry_dep, NH3_wet_dep

# -------------------------------------------------------------------
# 3.  Calendar-year totals
# -------------------------------------------------------------------
ds_yr = ds_mo.groupby("time.year").sum(dim="time")   # keeps lat, lon
ds_yr.to_netcdf(OUTFILE)
print("✓ wrote yearly NetCDF →", OUTFILE)

dry = ds_yr.NH3_dry_dep
wet = ds_yr.NH3_wet_dep

# Fixed colour scales (0–95 %)
vmin_d, vmax_d = 0, np.nanpercentile(dry, 95)
vmin_w, vmax_w = 0, np.nanpercentile(wet, 95)

# -------------------------------------------------------------------
# 4.  Plot loop by year
# -------------------------------------------------------------------
for yr in ds_yr.year.values:
    label = str(yr)

    # ---- dry map ----
    fig, ax = plt.subplots(figsize=(8, 4.5), dpi=120,
                            subplot_kw={"projection": ccrs.PlateCarree()})
    pcm = dry.sel(year=yr).plot(
        ax=ax, cmap="plasma", vmin=vmin_d, vmax=vmax_d,
        add_colorbar=False)
    ax.set_title(f"NH₃ Dry Deposition • {label}")
    ax.coastlines(linewidth=0.5); ax.add_feature(cfeature.BORDERS, linewidth=0.2)
    fig.colorbar(pcm, ax=ax, label="kg NH₃ m⁻² yr⁻¹")
    fig.tight_layout(); fig.savefig(ODRY / f"NH3_dry_{label}.png"); plt.close(fig)

    # ---- wet map ----
    fig, ax = plt.subplots(figsize=(8, 4.5), dpi=120,
                            subplot_kw={"projection": ccrs.PlateCarree()})
    pcm = wet.sel(year=yr).plot(
        ax=ax, cmap="viridis", vmin=vmin_w, vmax=vmax_w,
        add_colorbar=False)
    ax.set_title(f"NH₃ Wet Deposition • {label}")
    ax.coastlines(linewidth=0.5); ax.add_feature(cfeature.BORDERS, linewidth=0.2)
    fig.colorbar(pcm, ax=ax, label="kg NH₃ m⁻² yr⁻¹")
    fig.tight_layout(); fig.savefig(OWET / f"NH3_wet_{label}.png"); plt.close(fig)

print(f"✓ Plots saved → {ODRY}/ and {OWET}/")

✓ wrote yearly NetCDF → NH3_yearly_dep_2019-2022.nc
✓ Plots saved → fig_dry_yearly/ and fig_wet_yearly/


In [3]:
dry

<xarray.DataArray 'NH3_dry_dep' (year: 4, lat: 104, lon: 97)> Size: 323kB
array([[[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        ...,
        [ 9.40255830e-05, -7.15588796e-06,  1.85545217e-04, ...,
          1.84949350e-03,  1.00681839e-03,  1.69501487e-03],
        [ 3.95424852e-04,  7.47677574e-04, -9.23212151e-06, ...,
          4.50408310e-04,  1.15657782e-03,  1.33995757e-03],
        [-3.75923869e-04,  4.29829419e-04,  2.01268042e-04, ...,
          6.23053809e-04,  1.12170470e-03,  3.54131061e-04]],

       [[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
...
        [ 1.07509229e-05, -3.73038173e-05,  3.51991655e-05, ...,
          1.08881256e-03, -8.28370370e-05, -4.18013142e-04],
        [ 9.24659139e-05,  4.18717632e-06,  3.04881969e-05, ...,
          1.23149641e-03,  1.80358589e-03,  1.14164469e-03],
        [-2.02651394e-05, -6.08317148e-06, -9.71657740e-06, ...,
          1.13862798e-03,  1.11452699e-03,  1.63995926e-03]],

       [[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        ...,
        [-1.55879151e-06, -7.75423427e-06,  2.61390355e-05, ...,
          1.06832015e-04,  3.59050714e-04,  2.22981940e-04],
        [ 8.75535496e-06,  1.09782585e-05,  2.43096929e-05, ...,
          4.78742520e-04,  4.49512519e-04,  3.53699814e-04],
        [-7.04622404e-06,  3.31685991e-05, -1.90139096e-05, ...,
          7.98800734e-05,  8.34450697e-04,  2.16162872e-06]]])
Coordinates:
  * lon      (lon) float64 776B -9.713 -9.574 -9.435 ... 3.348 3.487 3.626
  * lat      (lat) float64 832B 33.36 33.47 33.58 33.69 ... 44.28 44.39 44.5
  * year     (year) int64 32B 2019 2020 2021 2022
Attributes:
    units:    kg NH₃ m⁻² month⁻¹

In [4]:
wet

<xarray.DataArray 'NH3_wet_dep' (year: 4, lat: 104, lon: 97)> Size: 323kB
array([[[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        ...,
        [ 1.96982843e-07, -2.21359799e-08, -4.64883471e-08, ...,
          1.01584976e-06,  2.13004057e-08, -4.00655850e-07],
        [-1.38484393e-08,  4.31270045e-08, -5.31873033e-08, ...,
          4.52303676e-07,  8.98050835e-08,  7.93821966e-08],
        [ 1.33054074e-07, -2.13265622e-08,  9.30646776e-08, ...,
          2.34321969e-07,  5.41350334e-07,  5.02386279e-07]],

       [[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
...
        [ 1.49282235e-09, -6.32381728e-09,  3.07109777e-10, ...,
          8.54050007e-07,  4.71360431e-07,  5.08575150e-07],
        [ 7.72984224e-09, -9.16203567e-09,  8.23897783e-09, ...,
          8.56331414e-07, -5.58057917e-07,  8.04189745e-08],
        [ 3.03893241e-09, -3.36750270e-09, -8.27834192e-09, ...,
         -4.68249625e-08,  1.34134528e-08, -1.43949098e-07]],

       [[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        ...,
        [-3.48977998e-09, -4.60954032e-08,  1.35258793e-08, ...,
         -5.12582076e-09,  9.10420799e-09,  9.34552150e-10],
        [ 8.83122705e-10, -1.35290511e-09,  6.39643613e-09, ...,
          3.36466608e-08,  5.15773199e-09,  4.73570868e-09],
        [-4.84321876e-10,  1.23201004e-08, -1.30938933e-08, ...,
         -2.21775860e-08, -3.98101627e-09, -1.11416864e-09]]])
Coordinates:
  * lon      (lon) float64 776B -9.713 -9.574 -9.435 ... 3.348 3.487 3.626
  * lat      (lat) float64 832B 33.36 33.47 33.58 33.69 ... 44.28 44.39 44.5
  * year     (year) int64 32B 2019 2020 2021 2022
Attributes:
    units:    kg NH₃ m⁻² month⁻¹

In [ ]:
annual = out[["NH3_dry_dep", "NH3_wet_dep"]].groupby("time.year").sum()